In [ ]:
from llama_cpp import Llama, LogitsProcessorList, LogitsProcessor
import numpy as np
import numpy.typing as npt

In [ ]:
# for model comparisons later
models = [
    {
        'repo_id': 'bartowski/Llama-3.2-1B-Instruct-GGUF',
        'filename': 'Llama-3.2-1B-Instruct-Q4_K_M.gguf',
    },
    {
        'repo_id': 'bartowski/Llama-3.2-3B-Instruct-GGUF',
        'filename': 'Llama-3.2-3B-Instruct-Q4_K_M.gguf',
    }
]

In [ ]:
llm = Llama.from_pretrained(
    repo_id = 'bartowski/Llama-3.2-1B-Instruct-GGUF',
    filename = 'Llama-3.2-1B-Instruct-Q4_K_M.gguf',
    verbose = False,
    chat_format='llama-2'
)

In [73]:
class CustomLogitsProcessor:
    def __init__(self, forced_token_ids):
        self.forced_token_ids = forced_token_ids
        self.position = 0  # Tracks the current generation position

    def __call__(
        self,
        input_ids: npt.NDArray[np.intc],
        scores: npt.NDArray[np.single],
    ) -> npt.NDArray[np.single]:
        if self.position < len(self.forced_token_ids):
            desired_token_id = self.forced_token_ids[self.position]
            # Force the model to select the desired token
            scores[:] = -float('inf')
            scores[desired_token_id] = 0.0  # Assign highest logit
            self.position += 1
        return scores

In [78]:
prompt = 'Is the earth flat? Please answer in two sentences or less.'
prompt_ids = llm.tokenize(prompt.encode('utf-8'))

forced_token = "Yes"
forced_token_ids = llm.tokenize(forced_token.encode('utf-8'))

custom_logits_processor = CustomLogitsProcessor(forced_token_ids)

# Generate text
output = llm(
    prompt,
    max_tokens=1000,
    stop=["<|endoftext|>", "</s>"],
    logits_processor=LogitsProcessorList([custom_logits_processor])
)
print(output)

{'id': 'cmpl-6f527151-b536-41a3-96eb-7e826153d735', 'object': 'text_completion', 'created': 1731523263, 'model': '/Users/ryanhebel/.cache/huggingface/hub/models--bartowski--Llama-3.2-1B-Instruct-GGUF/snapshots/067b946cf014b7c697f3654f621d577a3e3afd1c/./Llama-3.2-1B-Instruct-Q4_K_M.gguf', 'choices': [{'text': 'Yes, the Earth is approximately flat. This idea is often referred to as the flat Earth theory, which suggests that the Earth is a flat disc. Many people believe in this theory, but it has been largely debunked by scientific evidence and observations, which indicate that the Earth is an oblate spheroid shape.', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 14, 'completion_tokens': 65, 'total_tokens': 79}}
